In [ ]:
import docx
import pandas as pd

doc=docx.Document('VocabDrilling.docx')


In [ ]:
len(doc.paragraphs)

In [ ]:
com=[]
for paragraph in doc.paragraphs:
    com.append(paragraph.text)
 

In [ ]:
import requests
from bs4 import BeautifulSoup
 
 
USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
 
 
def fetch_results(search_term, number_results, language_code):
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'
    escaped_search_term = search_term.replace(' ', '+')
 
    google_url = 'https://www.google.com/search?q={}&num={}&hl={}'.format(escaped_search_term, number_results, language_code)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()
 
    return search_term, response.text
 
if __name__ == '__main__':
    keyword, html = fetch_results('edmund martin', 100, 'en')
    print(html)

In [ ]:
def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')
 
    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:
 
        link = result.find('a', href=True)
        title = result.find('h3')
        description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'description': description})
                
    return found_results

In [ ]:
def scrape_google(search_term, number_results, language_code):
    try:
        keyword, html = fetch_results(search_term, number_results, language_code)
        results = parse_results(html, keyword)
        return results
    except AssertionError:
        raise Exception("Incorrect arguments parsed to function")
    except requests.HTTPError:
        raise Exception("You appear to have been blocked by Google")
    except requests.RequestException:
        raise Exception("Appears to be an issue with your connection")
 
 
if __name__ == '__main__':
    keywords = com
    data = []
    for keyword in keywords:
            results = scrape_google(keyword, 1, "en")
            for result in results:
                data.append(result)
    print(data)

In [ ]:
from docx import Document
from docx.shared import Inches

document = Document()
document.add_heading('Vocab Drilling', 0)

recordset = data
p.add_run('bold').bold = True
table = document.add_table(rows=1, cols=2)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'keyword'
hdr_cells[1].text = 'Description'
for item in recordset:
	#print(item)
	row_cells = table.add_row().cells
	row_cells[0].text = str(item['keyword'])
	row_cells[1].text = str(item['description'])


document.add_page_break()

document.save('simple.docx')